In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install yt_dlp
!pip install ffmpeg
!mkdir raw-video
!mkdir compressed-video
!mkdir log

In [ ]:
#@title Imports all libraries
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account

from googleapiclient.http import MediaFileUpload

import yt_dlp
import ffmpeg
import subprocess
import sys


In [ ]:
#@title Config Authorize Google Drive API
# If modifying these scopes, delete the file token.json.
USE_SERVICE_ACCOUNT = False
SCOPES = ["https://www.googleapis.com/auth/drive"]
TOKEN_FILE = "token.json"
CREDENTIALS_FILE = "credentials.json"
FOLDER_ID = "1-3y0hToO4S8KhoWzlrqcGwjOsBWOKvoX"

In [ ]:
url = ""

output_filename = ''

# Specify the output path
output_path = f'raw-video/'

# Create YoutubeDL object with the desired options
ydl_opts = {
  'outtmpl': output_path + '%(title)s.%(ext)s',
}

def download_from_url(url):
    info_dict = ydl.extract_info(url, download=True)
    
    # Extract the actual file name from the template
    downloaded_file_name = info_dict['title'] + '.' + info_dict['ext']
    
    return downloaded_file_name

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    output_filename = download_from_url(url)
    
    print(output_filename)

In [ ]:
#@title Compress video
input_file = f'raw-video/{output_filename}'
output_file = f'compressed-video/{output_filename}'
progress_file = f'log/{output_filename}.txt'

# Set the desired video bitrate and quality (adjust as needed)
video_bitrate = '1000k'  # 1000k is just an example, you can adjust based on your requirements
video_quality = '31' # Range: 2 (best) - 31 (worst)
resolution = {
    '1080p':'1920:1080',
    '720p':'720:480',
    '480p':'640:480',
    '360p':'480:360',
    '240p':'426:240',
    '144p':'256:144'
} 

# FFmpeg command to compress the video using H.265 codec
ffmpeg_cmd = f'ffmpeg -i {input_file} -vf scale={resolution["480p"]} -b:v {video_bitrate} -vcodec libx265 -crf {video_quality} -preset veryfast -progress {progress_file} {output_file}'

# Run the FFmpeg command
subprocess.run(ffmpeg_cmd.split(), capture_output=True, text=True)


In [ ]:
#@title Authorize Google Drive API
creds = None
if USE_SERVICE_ACCOUNT:
    creds = service_account.Credentials.from_service_account_file(
        "service_account.json", scopes=SCOPES
    )
else:
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())
service = build("drive", "v3", credentials=creds)

In [ ]:
#@title List files
try:
    results = (
        service.files()
        .list(q=f"'{FOLDER_ID}' in parents and trashed = false", pageSize=10, fields="nextPageToken, files(id, name)")
        .execute()
    )
    items = results.get("files", [])
    if not items:
        print("No files found.")
    else:
        print("Files:")
        for item in items:
            print(f"{item['name']} ({item['id']})")
except HttpError as error:
    print(f"An error occurred: {error}")

In [ ]:
#@title copy local video to google drive

local_video_path = f'compressed-video/{output_filename}'
file_metadata = {"name": output_filename, "parents": [FOLDER_ID]}
media = MediaFileUpload(local_video_path, mimetype="video/mp4")
try:
    pass
    file = (
        service.files()
        .create(body=file_metadata, media_body=media, fields="id")
        .execute()
    )

    print(f"File ID: {file.get('id')}")
except HttpError as error:
    print(f"An error occurred: {error}")